In [1]:
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Okt
import os
# --- 데이터 준비 ---
# 1. 전처리된 텍스트 로드
preprocessed_text_dir = '법령_전처리_텍스트'
docs = []
law_names_for_topic = []
for filename in os.listdir(preprocessed_text_dir):
    if filename.endswith('.txt'):
        law_names_for_topic.append(filename[:-4])
        with open(os.path.join(preprocessed_text_dir, filename), 'r', encoding='utf-8') as f:
            docs.append(f.read())

# 2. 분석 대상 그룹 정의 (변호사 시험 과목 기준)
# (public_laws, civil_laws, criminal_laws 리스트가 메모리에 있다고 가정)
law_to_subject = {law: '공법' for law in public_laws}
law_to_subject.update({law: '민사법' for law in civil_laws})
law_to_subject.update({law: '형사법' for law in criminal_laws})

# --- 토픽 모델링 실행 ---
# 1. 한국어 명사 추출을 위한 벡터라이저 설정
okt = Okt()
vectorizer = CountVectorizer(tokenizer=okt.nouns, max_features=3000)

# 2. BERTopic 모델 초기화
# min_topic_size: 토픽을 구성하기 위한 최소 문서(법령) 수
topic_model = BERTopic(embedding_model="jhgan/ko-sbert-nli",
                       vectorizer_model=vectorizer,
                       min_topic_size=5,
                       verbose=True)

# 3. 모델 학습 및 토픽 추출
# 시간이 매우 오래 걸릴 수 있습니다. ⏳
topics, probs = topic_model.fit_transform(docs)

# 4. 결과 확인
print("\n--- BERTopic 분석 결과 ---")
# 가장 자주 등장하는 토픽들
print(topic_model.get_topic_info())

# 특정 토픽의 키워드 확인 (예: 0번 토픽)
print("\n--- 0번 토픽의 주요 키워드 ---")
print(topic_model.get_topic(0))

# --- 그룹별 토픽 분석 ---
# 1. 각 법령에 토픽 ID와 과목 정보 추가
df_topics = topic_model.get_document_info(docs, law_names_for_topic)
df_topics['과목'] = df_topics['Name'].map(law_to_subject).fillna('기타')

# 2. 각 과목 그룹별로 어떤 토픽이 주로 나타나는지 시각화
# 이 시각화는 Jupyter Notebook 환경에서 가장 잘 보입니다.
fig = topic_model.visualize_topics_by_class(
    df_topics,
    top_n_topics=10, # 상위 10개 토픽만 시각화
    custom_labels=True
)
fig.show() # 대화형 차트 출력

ModuleNotFoundError: No module named 'bertopic'